# Sun storm predictor - NASA Space Apps 

## Participants
- Felipe Saadi
- Gabriel Caetano
- Luiz Granville
- Marcelo Feitoza
- Pedro Munhoz

---

### Libraries installation

In [314]:
! pip install cdflib spacepy --quiet

---

## Importing libraries and data (.windMission)

In [315]:
import pandas as pd
import numpy as np
from spacepy import pycdf

windMission = pycdf.CDF("wi_h2_mfi_20220916_v03.cdf") # The Wind Mission’s Ion Parameters
dscovr = pycdf.CDF("dscovr_h0_mag_20220917_v01.cdf") # The DSCOVR Magnetic Field Data Sets, BD(t)


In [316]:
# get the keys where the number of rows is equal to 925882
windMissionKeys = [key for key in windMission.keys() if windMission[key].shape[0] == 925882]
windMissionKeys

['Epoch', 'Time_PB5', 'BF1', 'BGSM', 'BGSE', 'RANGE', 'SPC_MODE', 'MAG_MODE']

In [317]:
dscovr.keys()

KeysView(<CDF:
B1F1: CDF_REAL4 [86400]
B1GSE: CDF_REAL4 [86400, 3]
B1RTN: CDF_REAL4 [86400, 3]
B1SDF1: CDF_REAL4 [86400]
B1SDGSE: CDF_REAL4 [86400, 3]
B1SDRTN: CDF_REAL4 [86400, 3]
Epoch1: CDF_EPOCH [86400]
FLAG1: CDF_INT2 [86400]
NUM1_PTS: CDF_INT4 [86400]
RANGE1: CDF_INT2 [86400]
SENS: CDF_REAL4 [3, 8] NRV
Time1_PB5: CDF_INT4 [86400, 3]
ZERO: CDF_REAL4 [3, 8] NRV
format_time: CDF_CHAR*2 [3] NRV
label_bgse: CDF_CHAR*8 [3] NRV
label_brtn: CDF_CHAR*8 [3] NRV
label_bsdgse: CDF_CHAR*14 [3] NRV
label_bsdrtn: CDF_CHAR*14 [3] NRV
label_time: CDF_CHAR*27 [3] NRV
unit_time: CDF_CHAR*4 [3] NRV
>)

## Exibindo a descrição dos dados

In [318]:
windMission['AMPL1_I'].attrs

<zAttrList:
CATDESC: Inner Sensor Amplitude Correction (1 min) [CDF_CHAR]
DEPEND_0: Epoch1 [CDF_CHAR]
FIELDNAM: Inner Sensor Amplitude Correction (1 min) [CDF_CHAR]
FILLVAL: -1e+31 [CDF_FLOAT]
MONOTON: FALSE [CDF_CHAR]
SCALETYP: LINEAR [CDF_CHAR]
TIME_RES: 1 min [CDF_CHAR]
VAR_TYPE: metadata [CDF_CHAR]
>

In [319]:
windMission['AMPL1_I'].attrs

<zAttrList:
CATDESC: Inner Sensor Amplitude Correction (1 min) [CDF_CHAR]
DEPEND_0: Epoch1 [CDF_CHAR]
FIELDNAM: Inner Sensor Amplitude Correction (1 min) [CDF_CHAR]
FILLVAL: -1e+31 [CDF_FLOAT]
MONOTON: FALSE [CDF_CHAR]
SCALETYP: LINEAR [CDF_CHAR]
TIME_RES: 1 min [CDF_CHAR]
VAR_TYPE: metadata [CDF_CHAR]
>

In [320]:
windMission['RANGE'].attrs

<zAttrList:
CATDESC: Magnetometer Range [CDF_CHAR]
DEPEND_0: Epoch [CDF_CHAR]
FIELDNAM: Magnetometer Range [CDF_CHAR]
FILLVAL: -1 [CDF_INT4]
FORMAT: I1 [CDF_CHAR]
LABLAXIS: Range [CDF_CHAR]
MONOTON: FALSE [CDF_CHAR]
SCALEMAX: 1 [CDF_INT4]
SCALEMIN: 0 [CDF_INT4]
SCALETYP: LINEAR [CDF_CHAR]
TIME_RES: Variable [CDF_CHAR]
UNITS:   [CDF_CHAR]
VALIDMAX: 7 [CDF_INT4]
VALIDMIN: 0 [CDF_INT4]
VAR_TYPE: support_data [CDF_CHAR]
>

In [321]:
windMission['FLAG1_I'].attrs

<zAttrList:
CATDESC: Inner Sensor Zero Flag (1 min) [CDF_CHAR]
DEPEND_0: Epoch1 [CDF_CHAR]
FIELDNAM: Inner Sensor Zero Flag (1 min) [CDF_CHAR]
FILLVAL: -1 [CDF_INT4]
MONOTON: FALSE [CDF_CHAR]
SCALETYP: LINEAR [CDF_CHAR]
TIME_RES: 1 min [CDF_CHAR]
VAR_TYPE: metadata [CDF_CHAR]
>

In [322]:
windMission['MAG_MODE'].attrs

<zAttrList:
CATDESC: WIND/MFI operational mode [CDF_CHAR]
DEPEND_0: Epoch [CDF_CHAR]
FIELDNAM: WIND/MFI operational mode [CDF_CHAR]
FILLVAL: -1 [CDF_INT4]
FORMAT: I2 [CDF_CHAR]
LABLAXIS: MFI MODE [CDF_CHAR]
MONOTON: FALSE [CDF_CHAR]
SCALEMAX: 11 [CDF_INT4]
SCALEMIN: 11 [CDF_INT4]
SCALETYP: LINEAR [CDF_CHAR]
TIME_RES: Variable [CDF_CHAR]
UNITS:   [CDF_CHAR]
VALIDMAX: 15 [CDF_INT4]
VALIDMIN: 0 [CDF_INT4]
VAR_TYPE: support_data [CDF_CHAR]
>

windMission['Ma']

In [323]:
windMission['Epoch1'].attrs

<zAttrList:
CATDESC: Time, Centered, Number of milliseconds since the epoch (1 min) [CDF_CHAR]
FIELDNAM: Time Line (1 min) [CDF_CHAR]
FILLVAL: -1e+31 [CDF_DOUBLE]
FORMAT: E14.8 [CDF_CHAR]
LABLAXIS: Epoch [CDF_CHAR]
MONOTON: INCREASE [CDF_CHAR]
SCALEMAX: 63830592000000.0 [CDF_DOUBLE]
SCALEMIN: 63830505600000.0 [CDF_DOUBLE]
SCALETYP: LINEAR [CDF_CHAR]
TIME_RES: 1 min [CDF_CHAR]
UNITS: ms [CDF_CHAR]
VALIDMAX: 64092124800000.0 [CDF_DOUBLE]
VALIDMIN: 62951817600000.0 [CDF_DOUBLE]
VAR_TYPE: support_data [CDF_CHAR]
>

## Conversion to a common DataFrame

In [324]:
epoch = pd.DataFrame(windMission['Epoch']).rename(columns={0: 'Epoch'})
bgse = pd.DataFrame(windMission['BGSE']).rename(columns={0: 'Bx', 1: 'By', 2: 'Bz'})
bf1 = pd.DataFrame(windMission['BF1']).rename(columns={0: 'Magnetic field magnitude'})
stormRange = pd.DataFrame(windMission['RANGE']).rename(columns={0: 'Storm range'})
spc_mode = pd.DataFrame(windMission['SPC_MODE']).rename(columns={0: 'S/C operational mode'})
mag_mode = pd.DataFrame(windMission['MAG_MODE']).rename(columns={0: 'WIND/MFI operational mode'})

dataframe2 = [epoch, bgse, bf1, stormRange, spc_mode, mag_mode] 
df2 = pd.concat(dataframe2, axis=1)
df2.head()

,Epoch,Bx,By,Bz,Magnetic field magnitude,Storm range,S/C operational mode,WIND/MFI operational mode
0,2022-09-16 00:00:00.030,2.206256,-7.910860,6.915058,10.736261,1,1,11
1,2022-09-16 00:00:00.122,2.187438,-7.958672,6.887069,10.749747,1,1,11
2,2022-09-16 00:00:00.214,2.208745,-7.976111,6.875369,10.759536,1,1,11
3,2022-09-16 00:00:00.306,2.147695,-7.943528,6.880512,10.726308,1,1,11
4,2022-09-16 00:00:00.398,2.160743,-7.915912,6.860727,10.695796,1,1,11


In [325]:
# Turn the Epoch column into unix time
df2['Epoch'] = pd.to_datetime(df2['Epoch']).astype('int64') // 10**9
df2['Epoch'].head()

0    1663286400
1    1663286400
2    1663286400
3    1663286400
4    1663286400
Name: Epoch, dtype: int64

# Regressão linear - Target: Magnetic field magnitude

In [326]:
from sklearn.model_selection import train_test_split
x_entrada = df2[['Bx','By','Bz', 'Storm range', 'Epoch',
                'S/C operational mode', 'WIND/MFI operational mode']].values

y_saida = df2['Magnetic field magnitude'].values
# 'Epoch'
X_train, X_test, Y_train, Y_test = train_test_split(x_entrada, y_saida, 
                                                    test_size = 0.3, 
                                                    random_state = 42)

In [327]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_pred

array([6.10124825, 4.53865127, 6.67687571, ..., 4.75057543, 8.07562867,
       8.06289451])

In [328]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print('Acuracidade (treino): ', model.score(X_train, Y_train))
print('Acuracidade (teste): ', model.score(X_test, Y_test))
print('R2 score: ', r2_score(Y_test, Y_pred))
print('Mean squared error: ', mean_squared_error(Y_test, Y_pred))
print('Mean absolute error: ', mean_absolute_error(Y_test, Y_pred))

Acuracidade (treino):  0.8405036748829848
Acuracidade (teste):  0.8399662592589445
R2 score:  0.8399662592589445
Mean squared error:  0.7528603294930323
Mean absolute error:  0.6603896811709434


In [329]:
print(Y_pred)

[6.10124825 4.53865127 6.67687571 ... 4.75057543 8.07562867 8.06289451]


In [330]:
Y_pred2 = model.predict(X_train)
Y_pred2

array([4.86141295, 6.54016459, 9.64136956, ..., 8.82840611, 5.57693029,
       9.2463506 ])

In [331]:
Y_pred_total = np.concatenate((Y_pred2, Y_pred), axis=0)
print(len(Y_pred_total))

925882


In [332]:
df2['Prediction'] = Y_pred_total
df2.head()

,Epoch,Bx,By,Bz,Magnetic field magnitude,Storm range,S/C operational mode,WIND/MFI operational mode,Prediction
0,1663286400,2.206256,-7.910860,6.915058,10.736261,1,1,11,4.861413
1,1663286400,2.187438,-7.958672,6.887069,10.749747,1,1,11,6.540165
2,1663286400,2.208745,-7.976111,6.875369,10.759536,1,1,11,9.641370
3,1663286400,2.147695,-7.943528,6.880512,10.726308,1,1,11,10.240020
4,1663286400,2.160743,-7.915912,6.860727,10.695796,1,1,11,4.440167


## Saving model

In [338]:
import pickle
import joblib as jbl  

saved_model = pickle.dumps(model)
jbl.dump(model, 'model.pkl')
  
# Load the pickled model
modelPickle = pickle.loads(saved_model)
  
# Use the loaded pickled model to make predictions
modelPickle.predict(X_test)

df2.to_csv('dataframe2.csv', index=False)